In [ ]:
import io
import cv2
import base64
import requests
from PIL import Image


class ControlnetRequest:
    def __init__(self, url, prompt, path, seed, ctrlw=1.0):
        self.url = url
        self.prompt = prompt
        self.img_path = path
        self.body = None
        self.seed = seed
        self.ctrlw = ctrlw

    def build_body(self, ctrl=False):
        if ctrl:
            self.body = {
                "prompt": self.prompt,
                "negative_prompt": "",
                "batch_size": 1,
                "steps": 30,
                "cfg_scale": 7.5,
                "width": 512,
                "height": 512,
                "seed": self.seed,
                "alwayson_scripts": {
                    "controlnet": {
                        "args": [
                            {
                                "enabled": True,
                                "image": self.read_image(),
                                "module": "canny",
                                "model": "control_v11p_sd15_canny [d14c016b]",
                                # "model": "fusrs_v2_512_epoch_88 [fef5e48e]",
                                "resize_mode": 0,
                                "weight": self.ctrlw,
                                "lowvram": False,
                                "control_mode": 2,
                                # "processor_res": 1000,
                                "threshold_a": 150,
                                "threshold_b": 250,
                                "guidance_start": 0.0,
                                "guidance_end": 1.0,
                                "pixel_perfect": True,
                            }
                        ]
                    }
                },
            }
        else:
            self.body = {
                "prompt": self.prompt,
                "negative_prompt": "",
                "batch_size": 1,
                "steps": 30,
                "cfg_scale": 7.5,
                "width": 512,
                "height": 512,
                "seed": self.seed,
                "alwayson_scripts": {
                    "controlnet": {
                        "args": [
                            {
                                "enabled": False,
                            }
                        ]
                    }
                },
            }

    def send_request(self):
        response = requests.post(url=self.url, json=self.body)
        return response.json()

    def read_image(self):
        img = cv2.imread(self.img_path)
        retval, bytes = cv2.imencode(".png", img)
        encoded_image = base64.b64encode(bytes).decode("utf-8")
        return encoded_image

In [ ]:
import numpy as np


url = "http://localhost:7861/sdapi/v1/txt2img"
path = "./ors0.65.png"


# 0 weight
prompt = f"SAR image of tank"

control_net = ControlnetRequest(url, prompt, path, seed=100)  # Replace None if needed
control_net.build_body(ctrl=False)
output = control_net.send_request()

result1 = output["images"][0]
image1 = Image.open(
    io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
)  # Changed [0] to [1] to get the actual image data
print("0")
image1.show()

for weight in np.arange(0.0, 1.0, 0.1):  # This will iterate weights from 0.1 to 1.0
    prompt = f"SAR image of tank, <lora:sar:{weight}>"
    print(prompt)
    control_net = ControlnetRequest(
        url, prompt, path, seed=100
    )  # Replace None if needed
    control_net.build_body(ctrl=True)
    output = control_net.send_request()

    result1 = output["images"][0]
    image1 = Image.open(
        io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
    )  # Changed [0] to [1] to get the actual image data
    print(weight)
    image1.show()

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import io
import base64
from IPython.display import display

url = "http://localhost:7861/sdapi/v1/txt2img"
path = "./ors0.65.png"

def fetch_image(prompt):
    # I assume your ControlnetRequest, build_body, and send_request methods 
    # do what's necessary to fetch the image.
    control_net = ControlnetRequest(url, prompt, path, seed=100)
    control_net.build_body(ctrl=True)
    output = control_net.send_request()
    result = output["images"][0]
    return Image.open(io.BytesIO(base64.b64decode(result.split(",", 1)[0])))

# Dimensions of each small image and the number of images per row.
img_dim = 256
imgs_per_row = 4

# Create a blank white canvas to hold all images.
canvas = Image.new('RGB', (img_dim * imgs_per_row, img_dim * imgs_per_row), 'white')
draw = ImageDraw.Draw(canvas)

# Fetch the first image without any weight and place it on the canvas.
image = fetch_image("SAR image of tank")
image = image.resize((img_dim, img_dim))
canvas.paste(image, (0, 0))

for idx, weight in enumerate(np.arange(0.0, 1.0, 0.1)):
    prompt = f"SAR image of tank, <lora:sar:{weight}>"
    image = fetch_image(prompt)
    image = image.resize((img_dim, img_dim))
    
    # Calculate the position on the canvas.
    x = (idx + 1) % imgs_per_row * img_dim
    y = (idx + 1) // imgs_per_row * img_dim
    
    canvas.paste(image, (x, y))

# Display the canvas on Jupyter Notebook.
display(canvas)


In [ ]:
import numpy as np


url = "http://localhost:7861/sdapi/v1/txt2img"
path = "./ors0.65.png"


# 0 weight
prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.65>"

control_net = ControlnetRequest(url, prompt, path, seed=100)  # Replace None if needed
control_net.build_body(ctrl=False)
output = control_net.send_request()

result1 = output["images"][0]
image1 = Image.open(
    io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
)  # Changed [0] to [1] to get the actual image data
print("0")
image1.show()

for weight in np.arange(0.2, 1.0, 0.1):  # This will iterate weights from 0.1 to 1.0
    prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.65>"
    print(prompt)
    control_net = ControlnetRequest(
        url, prompt, path, seed=100, ctrlw=weight
    )  # Replace None if needed
    control_net.build_body(ctrl=True)
    output = control_net.send_request()

    result1 = output["images"][0]
    image1 = Image.open(
        io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
    )  # Changed [0] to [1] to get the actual image data
    print(weight)
    image1.show()

In [ ]:
import numpy as np


url = "http://localhost:7861/sdapi/v1/txt2img"
path = "./ors0.65.png"


# 0 weight
prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.65>"

control_net = ControlnetRequest(url, prompt, path, seed=100)  # Replace None if needed
control_net.build_body(ctrl=False)
output = control_net.send_request()

result1 = output["images"][0]
image1 = Image.open(
    io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
)  # Changed [0] to [1] to get the actual image data
print("0")
image1.show()

for weight in np.arange(0.0, 1.0, 0.1):  # This will iterate weights from 0.1 to 1.0
    prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.65>, <lora:sar:{weight}>"
    print(prompt)
    control_net = ControlnetRequest(
        url, prompt, path, seed=100, ctrlw=0.5
    )  # Replace None if needed
    control_net.build_body(ctrl=True)
    output = control_net.send_request()

    result1 = output["images"][0]
    image1 = Image.open(
        io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
    )  # Changed [0] to [1] to get the actual image data
    print(weight)
    image1.show()

In [ ]:
import numpy as np


url = "http://localhost:7861/sdapi/v1/txt2img"
path = "./ors0.65.png"


# 0 weight
prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.65>"

control_net = ControlnetRequest(url, prompt, path, seed=100)  # Replace None if needed
control_net.build_body(ctrl=False)
output = control_net.send_request()

result1 = output["images"][0]
image1 = Image.open(
    io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
)  # Changed [0] to [1] to get the actual image data
print("0")
image1.show()

for weight in np.arange(0.0, 1.0, 0.05):  # This will iterate weights from 0.1 to 1.0
    prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:{1-weight}>, <lora:sar:{weight}>"
    print(prompt)
    control_net = ControlnetRequest(
        url, prompt, path, seed=100, ctrlw=0.5
    )  # Replace None if needed
    control_net.build_body(ctrl=True)
    output = control_net.send_request()

    result1 = output["images"][0]
    image1 = Image.open(
        io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
    )  # Changed [0] to [1] to get the actual image data
    print(weight)
    image1.show()

In [ ]:
for weight in np.arange(1.0, 1.5, 0.1):  # This will iterate weights from 0.1 to 1.0
    prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.7>, <lora:sar:{weight}>"

    control_net = ControlnetRequest(
        url, prompt, path, seed=100
    )  # Replace None if needed
    control_net.build_body(ctrl=True)
    output = control_net.send_request()

    result1 = output["images"][0]
    image1 = Image.open(
        io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
    )  # Changed [0] to [1] to get the actual image data
    print(weight)
    image1.show()

In [ ]:
import numpy as np


url = "http://localhost:7861/sdapi/v1/txt2img"
path = "./ors0.65.png"


# 0 weight
prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.65>"

control_net = ControlnetRequest(url, prompt, path, seed=100)  # Replace None if needed
control_net.build_body(ctrl=False)
output = control_net.send_request()

result1 = output["images"][0]
image1 = Image.open(
    io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
)  # Changed [0] to [1] to get the actual image data
print("0")
image1.show()

for weight in np.arange(0.0, 1.0, 0.1):  # This will iterate weights from 0.1 to 1.0
    prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.65>, <lora:sar:{weight}>"
    print(prompt)
    control_net = ControlnetRequest(
        url, prompt, path, seed=100, ctrlw=0.5
    )  # Replace None if needed
    control_net.build_body(ctrl=True)
    output = control_net.send_request()

    result1 = output["images"][0]
    image1 = Image.open(
        io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
    )  # Changed [0] to [1] to get the actual image data
    print(weight)
    image1.show()

In [ ]:
import numpy as np


url = "http://localhost:7861/sdapi/v1/txt2img"
path = "./110.png"


# 0 weight
prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:0.65>"


control_net = ControlnetRequest(url, prompt, path, seed=1)  # Replace None if needed
control_net.build_body(ctrl=False)
output = control_net.send_request()

result1 = output["images"][0]
image1 = Image.open(
    io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
)  # Changed [0] to [1] to get the actual image data
print("0")
image1.show()

for weight in np.arange(0.0, 1.0, 0.05):  # This will iterate weights from 0.1 to 1.0
    prompt = f"ORS, SAR image of tanker ship, <lora:dosrs_ors:{1-weight}>, <lora:sar:{weight}>"
    print(prompt)
    control_net = ControlnetRequest(
        url, prompt, path, seed=1, ctrlw=0.5
    )  # Replace None if needed
    control_net.build_body(ctrl=True)
    output = control_net.send_request()

    result1 = output["images"][0]
    image1 = Image.open(
        io.BytesIO(base64.b64decode(result1.split(",", 1)[0]))
    )  # Changed [0] to [1] to get the actual image data
    print(weight)
    image1.show()